In [1]:
import numpy as np
import pandas as pd
import keras as ks

In [2]:
df = pd.read_csv("Churn_Modelling.csv")

In [3]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [5]:
df.duplicated().sum()

0

In [6]:
df['Exited'].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

In [7]:
df = df.drop(columns=['RowNumber','CustomerId','Surname'])

In [8]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
# performing one hot encoding in column of geography and gender
df = pd.get_dummies(df,columns=['Geography',"Gender"])


In [10]:
df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,True,False,False,True,False
1,608,41,1,83807.86,1,0,1,112542.58,0,False,False,True,True,False
2,502,42,8,159660.80,3,1,0,113931.57,1,True,False,False,True,False
3,699,39,1,0.00,2,0,0,93826.63,0,True,False,False,True,False
4,850,43,2,125510.82,1,1,1,79084.10,0,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,True,False,False,False,True
9996,516,35,10,57369.61,1,1,1,101699.77,0,True,False,False,False,True
9997,709,36,7,0.00,1,0,1,42085.58,1,True,False,False,True,False
9998,772,42,3,75075.31,2,1,0,92888.52,1,False,True,False,False,True


In [34]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
x = df.drop(columns=["Exited"])
y = df["Exited"]
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)

In [35]:
sc = StandardScaler()
x_train_scaled = sc.fit_transform(x_train)
x_test_scaled = sc.transform(x_test)

In [36]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [37]:
model = Sequential()
model.add(Dense(11,activation="relu"))
model.add(Dense(11,activation="relu"))
model.add(Dense(11,activation="relu"))
model.add(Dense(1,activation="relu"))

In [38]:
model.compile(loss="binary_crossentropy",optimizer="Adam")

In [39]:
model.fit(x_train_scaled,y_train,epochs=100)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.9691
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5047
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4565
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4478
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 979us/step - loss: 0.4404
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4214
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4269
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4057
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 893us/step - loss: 0.4104
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 963us/step - loss: 0.3956
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3915
Epoch 12/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 870us/step - loss: 0.3863
Epoch 13/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 853us/step - loss: 0.4523
Epoch 14/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 984us/step - loss: 0.3889
Epoch 15/100
250/250 ━━━━━━━━━━

In [40]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (32, 11)               │           154 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (32, 11)               │           132 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (32, 11)               │           132 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (32, 1)                │            12 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,292 (5.05 KB)

 Trainable params: 430 (1.68 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 862 (3.37 KB)

In [41]:
model.layers[1].get_weights()

[array([[-0.32046226,  0.30934218,  0.54156375,  0.163019  , -0.3196772 ,
         -0.5716513 , -1.1885834 ,  0.13043576,  0.56425095,  0.09857783,
         -0.12934332],
        [-0.13977833,  0.22077799, -0.7637114 ,  0.116884  , -0.1540365 ,
         -0.27491337,  0.03541562, -0.3534691 , -0.23768884, -0.753632  ,
          0.24757595],
        [ 0.1143808 , -0.44002974,  0.18389623,  0.2660028 ,  0.04962648,
          0.34177524, -0.78497255,  0.1300627 , -0.23983729, -0.21361436,
         -0.15021369],
        [-0.674651  , -0.00165592, -0.24956115,  0.01789029,  0.10940282,
          0.1380832 ,  0.28664032, -0.13906959,  0.11053327,  0.12449267,
         -0.72067237],
        [ 0.39644164,  0.38089862,  0.1254003 , -0.7565228 ,  0.45345926,
         -0.35212156, -0.29973823,  0.2007516 ,  0.44852775,  0.60107476,
          0.01318597],
        [-0.22674488, -0.14026476, -0.47023073,  0.04641126, -0.21599054,
          0.03800898,  0.15954116, -0.8762219 ,  0.40770417,  0.3340588

In [42]:
x_train_pred = model.predict(x_train_scaled)
x_test_pred = model.predict(x_test_scaled)

250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 788us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [43]:
x_train_pred = np.where(x_train_pred>0.5,1,0)
x_test_pred = np.where(x_test_pred>0.5,1,0)

In [44]:
from sklearn.metrics import accuracy_score
print("training accuracy : ")
print(accuracy_score(y_train,x_train_pred))
print("testing accuracy : ")
print(accuracy_score(y_test,x_test_pred))


training accuracy : 
0.867
testing accuracy : 
0.8635


In [45]:
history = model.fit(x_train_scaled,y_train,epochs=100)


Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3202   
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3412
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 948us/step - loss: 0.3124
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3380
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 963us/step - loss: 0.3266
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 934us/step - loss: 0.3442
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 925us/step - loss: 0.3441
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3214 
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3330
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3291
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3255
Epoch 12/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3147
Epoch 13/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3134
Epoch 14/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.3366
Epoch 15/100
250/250 ━━━━━━━━━━

In [46]:
import matplotlib.pyplot as plt


In [53]:
# plt.plot(history.history[]])
# history.history

KeyError: 'accuracy'